In [1]:
import os
import pickle
import shutil
import shapely
import cartopy.crs as ccrs
import cartopy.geodesic as cgeo
import cartopy.vector_transform as cvt
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from shapely import prepare, Point, Polygon
from shapely.validation import make_valid
from skimage import measure, morphology
import sqlite3
from oceannavigator.dataset_config import DatasetConfig
from data import open_dataset
from data.sqlite_database import SQLiteDatabase

In [19]:
dataset_keys= DatasetConfig.get_datasets()
for dataset in dataset_keys:
    try:
        config= DatasetConfig(dataset)
        url= config.url
        with SQLiteDatabase(url) as db:
            variable_list= db.get_data_variables()
            variable=variable_list[0].key
            timestamp=db.get_latest_timestamp(variable)
            files = db.get_netcdf_files([timestamp], [variable])
            src_path = files[-1]
            filename = os.path.basename(src_path)
            if not os.path.exists(filename):
                shutil.copy(src_path, filename)
            ds= xr.open_dataset(filename)
            var=ds[variable]
        # get surface level data
        if len(ds.dims)==3:
            surface_data = var[0, :, :].data
        elif len(ds.dims)==2:
            surface_data= var.data
        else:
            surface_data = var[0, 0, :, :].data

        # create binary mask from data
        binary_mask = np.where(np.isnan(surface_data), 0, 1)
        binary_mask = np.pad(binary_mask, 1)  # pad the mask so that the edges will be included in the perimeter

        # create a convex hull mask
        ch_mask = morphology.convex_hull_image(binary_mask)

        # get the contours from the mask
        contours = measure.find_contours(ch_mask, level=0)

        # select the first contour for our perimeter (the first element should be the one we're interested in but you'll have to confirm yourself)
        perim_y, perim_x = np.transpose(contours[0]).astype(int)

        # shift coordinates on array edges so that we're not selecting the padded portion
        height, width = ch_mask.shape

        perim_y[perim_y == 0] = 1
        perim_y[perim_y >= height - 1] = height - 2

        perim_x[perim_x == 0] = 1
        perim_x[perim_x >= width - 1] = width - 2

        #second version
        lat_var = ds.get("latitude", ds.get("lat"))
        lon_var=ds.get("longitude", ds.get("lon"))
        dim = lat_var.ndim
        # Select that actual lon lat values
        if dim==2:
            pad_lat = np.pad(lat_var.data, 1)
            pad_lon = np.pad(lon_var.data, 1)
            lon_mesh = lon_var.data
            lat_mesh = lat_var.data

        elif dim==1:
            lon_mesh, lat_mesh = np.meshgrid(ds.longitude.data, ds.latitude.data)
            pad_lat=np.pad(lat_mesh,1)
            pad_lon=np.pad(lon_mesh,1)


        pts = np.stack([lon_mesh, lat_mesh], axis=2)
        pts = np.apply_along_axis(lambda pt: Point(pt), 2, pts)
        perim_lat = pad_lat[perim_y, perim_x]
        perim_lon = pad_lon[perim_y, perim_x]
        perim_poly = Polygon(np.stack([perim_lon, perim_lat], axis=1))
        prepare(perim_poly)
        poly_mask = perim_poly.contains_properly(pts).astype(bool)

        #third version

        idx = np.argmax(np.abs(np.diff(perim_lon)))
        if shapely.is_simple(perim_poly)==False:
            new_lons = [360, 360, 0, 0]
            new_lats = [perim_lat[idx], 90, 90, perim_lat[idx + 1]]
            perim_lon = np.insert(perim_lon, idx + 1, new_lons)
            perim_lat = np.insert(perim_lat, idx + 1, new_lats)

        name = config.name.strip().replace(" ", "_") + ".pkl"
        with open(name, "wb") as f:
            pickle.dump(perim_poly, f)
    except Exception as e:
        print(f"Error: {e} for dataset: {dataset}")


Error: list index out of range for dataset: giops_fc_10d_2dll
Error: [Errno 2] No such file or directory: '/data/hpfx.collab.science.gc.ca/20250612/WXO-DD/model_riops/netcdf/forecast/polar_stereographic/2d/12/040/20250612T12Z_MSC_RIOPS_VOTEMPER_DBS-0.5m_PS5km_P040.nc' for dataset: riops_fc_2dps
Error: [Errno 2] No such file or directory: '/data/hpfx.collab.science.gc.ca/20250723/WXO-DD/model_ciops/salish-sea/500m/00/044/20250723T00Z_MSC_CIOPS-SalishSea_SeaWaterSalinity_DBS-all_LatLon0.008x0.005_PT044H.nc' for dataset: ciops-salish-sea_fc_3dll
Error: [Errno 2] No such file or directory: '/data/hpfx.collab.science.gc.ca/20250723/WXO-DD/model_ciops/salish-sea/500m/00/044/20250723T00Z_MSC_CIOPS-SalishSea_SeaSfcHeight_Sfc_LatLon0.008x0.005_PT044H.nc' for dataset: ciops-salish-sea_fc_2dll
Error: unable to open database file for dataset: caps_fc_3drp
Error: unable to open database file for dataset: caps_fc_2drp
Error: unable to open database file for dataset: gdwps_2dll
Error: too many indice

In [ ]:
with open(config.name, "wb") as f:
    pickle.dump(perim_poly, f)

In [18]:
dataset_keys= DatasetConfig.get_datasets()
config= DatasetConfig(dataset_keys[2])
with open(config.name, "rb") as f:
    poly = pickle.load(f)

point = Point([-65, 79])

print(poly.contains(point))
print(poly.boundary)

True
LINESTRING (-65 79.9, -65.1 79.9, -65.2 79.9, -65.3 79.9, -65.4 79.9, -65.5 79.9, -65.6 79.9, -65.7 79.9, -65.8 79.9, -65.9 79.9, -66 79.9, -66.1 79.9, -66.2 79.9, -66.3 79.9, -66.4 79.9, -66.5 79.9, -66.6 79.9, -66.7 79.9, -66.8 79.9, -66.9 79.9, -67 79.9, -67.1 79.9, -67.2 79.9, -67.3 79.9, -67.4 79.9, -67.5 79.9, -67.6 79.9, -67.7 79.9, -67.8 79.9, -67.9 79.9, -68 79.9, -68.1 79.9, -68.2 79.9, -68.3 79.9, -68.4 79.9, -68.5 79.9, -68.6 79.9, -68.7 79.9, -68.8 79.9, -68.9 79.9, -69 79.9, -69.1 79.9, -69.2 79.9, -69.3 79.9, -69.4 79.9, -69.5 79.9, -69.6 79.9, -69.7 79.9, -69.8 79.9, -69.9 79.9, -70 79.9, -70.1 79.9, -70.2 79.9, -70.3 79.9, -70.4 79.9, -70.5 79.9, -70.6 79.9, -70.7 79.9, -70.8 79.9, -70.9 79.9, -71 79.9, -71.1 79.9, -71.2 79.9, -71.3 79.9, -71.4 79.9, -71.5 79.9, -71.6 79.9, -71.7 79.9, -71.8 79.9, -71.9 79.9, -72 79.9, -72.1 79.9, -72.2 79.9, -72.3 79.9, -72.4 79.9, -72.5 79.9, -72.6 79.9, -72.7 79.9, -72.8 79.9, -72.9 79.9, -73 79.9, -73.1 79.9, -73.2 79.9, -73.3

In [2]:
dataset_keys= DatasetConfig.get_datasets()

In [5]:
config=dataset_keys[2]
config

'riops_fc_2dll'